#### PROCESS
- Data Import 
- Preprocessing for Business Logic
- Preprocessing for Model
- Modeling
- Training
- Validation 
- Inference
- Postprocessing
- Packaging
- Serving
- Monitoring

In [1]:
import os 
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from collections import defaultdict

from typing import List, Tuple

In [2]:
os.getcwd()
os.chdir('..')

In [3]:
ratings = pd.read_csv('dataset/ratings.csv').reset_index(drop=True)

In [4]:
user_num = ratings['userId'].nunique()
item_num = ratings['movieId'].nunique()
interaction_num = len(ratings)
sparsity = 1 - interaction_num / (user_num * item_num)
print('User number: %d, Item number: %d, Interaction number: %d, Sparsity: %.4f' % (user_num, item_num, interaction_num, sparsity))

User number: 610, Item number: 9724, Interaction number: 100836, Sparsity: 0.9830


In [5]:
# 유저 한 명당 몇 개의 아이템을 소비했는가?
ratings.groupby('userId').size().describe()

count     610.000000
mean      165.304918
std       269.480584
min        20.000000
25%        35.000000
50%        70.500000
75%       168.000000
max      2698.000000
dtype: float64

In [6]:
ratings['rating'].value_counts(normalize=True).sort_index(ascending=False)

rating
5.0    0.131015
4.5    0.084801
4.0    0.265957
3.5    0.130271
3.0    0.198808
2.5    0.055040
2.0    0.074884
1.5    0.017762
1.0    0.027877
0.5    0.013586
Name: proportion, dtype: float64

#### PROCESS
- Configuration
- Data Import 
- Preprocessing for Business Logic
- Preprocessing for Model
- Training
- Validation 
- Inference
- Postprocessing
- Packaging
- Serving
- Monitoring

#### CONFIGURATION
- 서비스에 필요한 모든 config를 제어할 수 있게 만드는 파트
- dev - stage - live 등의 환경 분리
- 모델 하이퍼 파라미터 관리
- 서비스 관련된 파라미터 관리

In [7]:
config = {
    'num_epoch': 10,
    'batch_size': 256,
    'lr': 0.001,
    'topk': 10,
    'valid_sample_size' : 10, 
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

##### Data Import
- 학습 데이터를 import 하는 단계
- 학습 데이터를 만들어주는 DE 작업(Batch, Feature Store 등..)이 필요함

In [8]:
ratings = pd.read_csv('dataset/ratings.csv').reset_index(drop=True)
# side information 적용 시 사용
movies = pd.read_csv('dataset/movies.csv').reset_index(drop=True)

##### Preprocessing For Business Logic
- 비즈니스 로직과 관련된 전처리 작업
  - e.g. 5점 척도를 0,1로 만들건데 3점 이상은 1로 처리할까요?
  - e.g. 어뷰저의 명단이 있다면 제외하는 것이 어때요?
  - e.g. 너무 인기없는 아이템은 제외할까요? 


In [9]:
# columns standardization
ratings.rename(columns={'userId': 'user_id', 'movieId': 'item_id'}, inplace=True)

In [10]:
# 3점 이상을 positive(=1)로, 3점 미만을 negative(=0)로
ratings['interaction'] = np.where(ratings['rating'] >= 3, 1, 0)

In [11]:
ratings['interaction'].value_counts(normalize=True)

interaction
1    0.810851
0    0.189149
Name: proportion, dtype: float64

##### Preprocessing For Model
- 모델의 input을 가공하기 위한 전처리 작업
  - Encoding / Embedding?
  - side information 포함 여부
- Dataset 인스턴스를 만들어서 DataLoader에 주입
  - Model에게 데이터를 어떻게 전달할 것인지 정의
  - Train Dataset과 Validation Dataset을 구분

In [12]:
class AEDataSet(Dataset):
    def __init__(self, ratings):
        self.ratings = ratings
        self.user_num = ratings['user_id'].nunique()
        self.item_num = ratings['item_id'].nunique()
        self.users = [i for i in range(self.user_num)]
        self.user2idx = self._encode(ratings, 'user_id')
        self.idx2user = self._decode(ratings, 'user_id')
        self.item2idx = self._encode(ratings, 'item_id')
        self.idx2item = self._decode(ratings, 'item_id')

        self.ratings['user_idx'] = self.ratings['user_id'].map(self.user2idx)
        self.ratings['item_idx'] = self.ratings['item_id'].map(self.item2idx)

        self.user_item_dict = self.get_user_item_dict()
        self.train_dict, self.valid_dict = self.train_valid_split()

            
    def _encode(self, df:pd.DataFrame, feature:str):
        """
        feature를 index로 변환하는 함수
        """
        return {val: idx for idx, val in enumerate(df[feature].unique())}
    
    def _decode(self, df:pd.DataFrame, feature:str):
        """
        index를 feature로 변환하는 함수
        """
        return {idx: val for idx, val in enumerate(df[feature].unique())}
    
    def get_user_item_dict(self):
        """
        user-item dictionary를 만드는 함수
        """
        user_item_dict = defaultdict(list)
        for _, row in self.ratings.iterrows():
            user_idx = int(row['user_idx'])
            item_idx = int(row['item_idx']) 
            user_item_dict[user_idx].append(item_idx)
        return user_item_dict
    
    def train_valid_split(self, valid_sample_size=10, seed=42):
        """
        train, valid 데이터셋을 나누는 함수
        """
        np.random.seed(seed)
        train = {}
        valid = {}
        for user, item in self.user_item_dict.items():
            valid_item = list(np.random.choice(item, valid_sample_size, replace=False, ))
            train_item = list(set(item) - set(valid_item))
            train[user] = train_item
            valid[user] = valid_item
        return train, valid

    def get_matrix(self, user_list:List[int], trainyn:bool=True):
        """
        AutoEncoder 모델에 입력할 데이터를 만들기 위한 함수
        """
        mat = torch.zeros((len(user_list), self.item_num))
        for idx, user in enumerate(user_list):
            if trainyn:
                mat[idx, self.train_dict[user]] = 1
            else:
                mat[idx, self.train_dict[user] + self.valid_dict[user]] = 1
        return mat
    
    def __len__(self):
        """
        DataLoader에서 데이터셋의 크기를 구하기 위한 함수
        """
        return len(self.users)
    
    def __getitem__(self, idx:int):
        """
        DataLoader에서 index를 통해 데이터를 불러오기 위한 함수
        """
        return self.users[idx]
    

#### MODELING
- 주어진 문제를 잘 해석하고 해결하는 모델을 만드는 단계

In [13]:
class AutoEncoder(nn.Module):
    def __init__(self, user_num:int, item_num:int, hidden_dim:int):
        super(AutoEncoder, self).__init__()
        self.user_num = user_num
        self.item_num = item_num
        self.hidden_dim = hidden_dim
        self.encoder = torch.nn.Linear(item_num, hidden_dim)
        self.decoder = torch.nn.Linear(hidden_dim, item_num)
        
    def forward(self, x):
        x = torch.sigmoid(self.encoder(x))
        x = self.decoder(x)
        return x

In [14]:
dataset = AEDataSet(ratings)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
model = AutoEncoder(user_num=dataset.user_num, item_num=dataset.item_num, hidden_dim=config['batch_size'])

In [15]:
# I/O 실험
for idx, user in enumerate(dataloader):
    user = user.tolist()
    mat = dataset.get_matrix(user, trainyn=False)
    output = model(mat)
    print(output)
    break

tensor([[-0.5224, -0.1163,  0.2507,  ..., -0.2362,  0.0164,  0.2071],
        [-0.5158, -0.1241,  0.2517,  ..., -0.2283,  0.0133,  0.1998],
        [-0.5103, -0.1217,  0.2482,  ..., -0.2341,  0.0120,  0.2064],
        ...,
        [-0.5203, -0.1394,  0.2582,  ..., -0.2470,  0.0171,  0.2188],
        [-0.5162, -0.1293,  0.2596,  ..., -0.2358,  0.0100,  0.1923],
        [-0.5004, -0.1155,  0.2841,  ..., -0.2161,  0.0232,  0.1940]],
       grad_fn=<AddmmBackward0>)


#### TRAINING & VALIDATION
- 지표(METRICS) 설정
- TRAIN과 VALIDATE를 반복하며 최적화 (하이퍼 파라미터, 모델 학습 주기 등)

In [16]:
### METRICS ### 
def get_ndcg(pred_list, true_list, k=config['topk']):
    def dcg(scores):
        return np.sum([rel / np.log2(idx + 2) for idx, rel in enumerate(scores)])

    relevance_scores = [1 if pred in true_list else 0 for pred in pred_list[:k]]

    ideal_scores = [1] * min(len(true_list), k)
    
    dcg_value = dcg(relevance_scores)
    idcg_value = dcg(ideal_scores)
    
    return dcg_value / idcg_value if idcg_value > 0 else 0

def get_hit(pred_list, true_list):
    hit_list = set(true_list) & set(pred_list)
    hit = len(hit_list) / len(true_list)
    return hit

def train(model, dataloader, dataset, optimizer, criterion, epoch):
    model.train()
    for idx, users in enumerate(dataloader):
        users = users.tolist()
        mat = dataset.get_matrix(users, trainyn=False)
        output = model(mat)
        loss = criterion(output, mat)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Epoch: %d, Loss: %.4f' % (epoch+1, loss.item()))

def evalutate(model, dataloader, dataset):
    NDCG = 0
    HIT = 0 
    with torch.no_grad():
        for idx, users in enumerate(dataloader):
            users = users.tolist()
            mat = dataset.get_matrix(users, trainyn=True)
            output = model(mat)
            output = torch.softmax(output, dim=1)
            output[mat == 1] = -1
            rec_list = output.argsort(dim = 1)
            for user, rec in zip(users, rec_list):
                pred_list = rec[-config['topk']:].tolist()
                true_list = dataset.valid_dict[user]
                ndcg = get_ndcg(pred_list, true_list)
                hit = get_hit(pred_list, true_list)
                NDCG += ndcg
                HIT += hit
    NDCG = NDCG / len(dataloader.dataset)
    HIT = HIT / len(dataloader.dataset)
    print('NDCG: %.4f, HIT: %.4f' % (NDCG, HIT))
    return NDCG, HIT



In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
for epoch in range(config['num_epoch']):
    train(model, dataloader, dataset, optimizer, criterion, epoch)
    evalutate(model, dataloader, dataset)

Epoch: 1, Loss: 347.3615
NDCG: 0.1078, HIT: 0.1179
Epoch: 2, Loss: 2114.5432
NDCG: 0.1296, HIT: 0.1408
Epoch: 3, Loss: 301.1130
NDCG: 0.1392, HIT: 0.1516
Epoch: 4, Loss: 190.2558
NDCG: 0.1550, HIT: 0.1689
Epoch: 5, Loss: 354.0776
NDCG: 0.1706, HIT: 0.1869
Epoch: 6, Loss: 4388.2959
NDCG: 0.1875, HIT: 0.2079
Epoch: 7, Loss: 167.9472
NDCG: 0.2020, HIT: 0.2215
Epoch: 8, Loss: 191.7677
NDCG: 0.2226, HIT: 0.2441
Epoch: 9, Loss: 374.8484
NDCG: 0.2371, HIT: 0.2595
Epoch: 10, Loss: 3298.1851
NDCG: 0.2425, HIT: 0.2690


#### INFERENCE & POSTPROCESSING
- 서비스에 올라간 모델이 추론하는 파트
- API 정의 (w/BE)
    - request : user의 uid
    - response : recommendation item 
    - 실제론 더 복잡!
- 후처리 작업
    - e.g. 판매 금지 리스트(셀러 사이드에서 올라온 부적절한 상품 등)
    - e.g. 추가 튜닝 - 추천 품질을 올리기 위한 re-ranking 등

In [19]:
def inference(model, dataloader, dataset, user_id):
    with torch.inference_mode():
        user = dataset.user2idx[user_id]
        mat = dataset.get_matrix([user], trainyn=False)
        output = model(mat)
        output = torch.softmax(output, dim=1)
        output[mat == 1] = -1
        rec_list = output.argsort(dim = 1)
        pred_list = rec_list[0][-config['topk']:].tolist()
        return [dataset.idx2item[idx] for idx in pred_list] 
    
random_user = np.random.choice(ratings['user_id'].unique())
inference(model, dataloader, dataset, random_user)

[3681, 1079, 1270, 2628, 1266, 2470, 3361, 2321, 2100, 913]

#### PACKAGING & SERVING
- MLOps 관점에서 모델을 패키징하고 서빙하는 파이프라인을 만드는 작업

#### MONITORING
- 추천이 서비스에서 잘 이루어지고 있는지 모니터링
- e.g. API에서 에러가 나지 않았는 지
- e.g. 추천이 제공되고 있는 아이템들은 적절한 지
- e.g. 지표가 하락하지 않았는 지